In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from textblob import TextBlob
from tweepy import Stream
from bs4 import BeautifulSoup
import requests
import pprint
import json
import pandas as pd
import pymongo
from sqlalchemy import func
from nltk.stem import WordNetLemmatizer
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
import pickle  
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

C:\Users\sande\Anaconda3\envs\pythondata\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sande\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
file1 = "data/combined.csv"
df = pd.read_csv(file1)

In [38]:
# We have to combine the CVS files and then include a column 'isalive'
# For the given date, the corresponding corresponding character will have 0 - Alive or 1 - Dead
df["isalive"] = "0"
for index, row in df.iterrows():
    if row["char"] == "Arya Stark":
        df.loc[index, "isalive"] = '0'
    if row["char"] == "Bran Stark":
        df.loc[index, "isalive"] = '0'
    if row["char"] == "Brienne of Tarth":
        df.loc[index, "isalive"] = '0'
    if row["char"] == "Cersei Lannister" :
        if row["date"] == "05/12/2019" \
            or row["date"] == "05/13/2019" \
            or row["date"] == "05/14/2019" \
            or row["date"] == "05/15/2019" \
            or row["date"] == "05/16/2019" \
            or row["date"] == "05/17/2019":
            df.loc[index, "isalive"] = '1'
        else:
            df.loc[index, "isalive"] = '0'
    if row["char"] == "Daenerys Targaryen":
        df.loc[index, "isalive"] = '0'
    if row["char"] == "Euron Greyjoy":
        if row["date"] == "05/12/2019" \
            or row["date"] == "05/13/2019" \
            or row["date"] == "05/14/2019" \
            or row["date"] == "05/15/2019" \
            or row["date"] == "05/16/2019" \
            or row["date"] == "05/17/2019":
            df.loc[index, "isalive"] = '1'
    if row["char"] == "Jaime Lannister":
        if row["date"] == "05/12/2019" \
            or row["date"] == "05/13/2019" \
            or row["date"] == "05/14/2019" \
            or row["date"] == "05/15/2019" \
            or row["date"] == "05/16/2019" \
            or row["date"] == "05/17/2019":
            df.loc[index, "isalive"] = '1'
    if row["char"] == "Jon Snow":
        df.loc[index, "isalive"] = '0'
    if row["char"] == "Missandei":
        df.loc[index, "isalive"] = '0'
    if row["char"] == "Sandor Clegane (“The Hound”)":
        if row["date"] == "05/12/2019" \
            or row["date"] == "05/13/2019" \
            or row["date"] == "05/14/2019" \
            or row["date"] == "05/15/2019" \
            or row["date"] == "05/16/2019" \
            or row["date"] == "05/17/2019":
            df.loc[index, "isalive"] = '1'
    if row["char"] == "Sansa Stark":
        df.loc[index, "isalive"] = '0'
    if row["char"] == "Theon Greyjoy":
        if row["date"] == "05/12/2019" \
            or row["date"] == "05/13/2019" \
            or row["date"] == "05/14/2019" \
            or row["date"] == "05/15/2019" \
            or row["date"] == "05/16/2019" \
            or row["date"] == "05/17/2019":
            df.loc[index, "isalive"] = '1'
    if row["char"] == "Tyrion Lannister":
        df.loc[index, "isalive"] = '0'

In [39]:
df_backup = df

In [92]:
df = df_backup
df.head()

,text,date,char,sentiment,polarity,subjectivity,timestamp,Preprocess,isalive
0,“What do we say to the God of Death?” “Not tod...,05-11-2019,Arya Stark,Neu,0.000000,0.000000,05-11-2019 23:58,what do we say to the god of death not today i...,0
1,FUNKO POP GAME OF THRONES GOT ARYA STARK NYMER...,05-11-2019,Arya Stark,Neg,-0.116667,0.616667,05-11-2019 23:57,funko pop game of throne got arya stark nymeri...,0
2,@Annendil Arya Stark taught me how to do it.,05-11-2019,Arya Stark,Neg,-0.200000,0.600000,05-11-2019 23:55,annendil arya stark taught me how to do it,0
3,Taylor Swift reveals how Reputation album was ...,05-11-2019,Arya Stark,Neg,-0.400000,0.400000,05-11-2019 23:55,taylor swift reveals how reputation album wa i...,0
4,@bheraldo7 @qwexi_pablo Arya Stark #GameofThrones,05-11-2019,Arya Stark,Neg,-0.200000,0.600000,05-11-2019 23:53,bheraldo7 qwexi_pablo arya stark gameofthrones,0


In [93]:
df2 = df.groupby(["char", "date", "isalive"]).agg({"polarity" : "mean", "subjectivity" : "mean"})
# df2 = df.groupby(["char", "date", "isalive"])["polarity"].mean()
df = df2.reset_index()
df.head()

,char,date,isalive,polarity,subjectivity
0,Arya Stark,05-11-2019,0,-0.037314,0.511225
1,Arya Stark,05-12-2019,0,-0.054244,0.469481
2,Arya Stark,05/13/2019,0,-0.157228,0.516924
3,Arya Stark,05/14/2019,0,-0.077106,0.513569
4,Arya Stark,05/15/2019,0,-0.068706,0.435747


In [94]:
df3 = pd.DataFrame()
# df["polarity"]

In [95]:
df3["char"] = [char for char in df["char"]]
df3["date"] = [date for date in df["date"]]
df3["isalive"] = [isalive for isalive in df["isalive"]]
df3["polarity"] = [polarity for polarity in df["polarity"]]
df3["subjectivity"] = [subjectivity for subjectivity in df["subjectivity"]]
# df4 = pd.DataFrame({"char": df3["char"],
#                     "date": df3["date"],
#                     "isalive": df3["isalive"],
#                     "polarity": df3["polarity"]
#                    })

In [96]:
df = df3
df.head()

,char,date,isalive,polarity,subjectivity
0,Arya Stark,05-11-2019,0,-0.037314,0.511225
1,Arya Stark,05-12-2019,0,-0.054244,0.469481
2,Arya Stark,05/13/2019,0,-0.157228,0.516924
3,Arya Stark,05/14/2019,0,-0.077106,0.513569
4,Arya Stark,05/15/2019,0,-0.068706,0.435747


In [97]:
# df.head()
df1 = df.loc[(df["date"] == "05/11/2019") | (df["date"] == "05/12/2019")]
# y = [y for y in df1["isalive"]]
y = df1["isalive"]
# y

7     0
8     0
14    0
15    0
21    0
22    1
28    0
29    0
35    0
36    1
41    0
42    1
48    0
49    0
55    0
61    0
62    1
68    0
69    0
75    0
76    1
82    0
83    0
Name: isalive, dtype: object

In [63]:
# df2["polarity"] = [i for i in df1["polarity"]]
# df2["subjectivity"] = [i for i in df1["subjectivity"]]

In [84]:
# df1["polarity"]

7     0.066691
8     0.032667
14    0.078799
15    0.071109
21    0.035133
22    0.599365
28    0.427090
29    0.439957
35   -0.026238
36   -0.104189
41    0.017441
42    0.009998
48   -0.137149
49   -0.028366
55    0.000354
61    0.076310
62    0.113423
68   -0.076558
69   -0.082730
75   -0.007188
76   -0.000588
82   -0.006562
83    0.001464
Name: polarity, dtype: float64

In [99]:
X[0]

[0.0666909016909017, 0.21711741961741962]

In [98]:
X = []
for index, row in df1.iterrows():
    x = []
    x.append(row["polarity"])
    x.append(row["subjectivity"])
    X.append(x)

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, test_size=0.5)

In [101]:
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [102]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [103]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8181818181818182
Testing Data Score: 0.75


In [ ]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

In [ ]:
# df2 = df.loc[(df["date"] != "05/11/2019") & (df["date"] != "05/12/2019")]
df2 = df.loc[df["date"] == "05/17/2019"]
new_data = []
for index, row in df2.iterrows():
    x = []
    x.append(row["polarity"])
    x.append(row["subjectivity"])
    new_data.append(x)

In [ ]:
predictions = classifier.predict(new_data)
pred_df = pd.DataFrame({"Prediction": predictions})

In [ ]:
pred_df["Prediction"].nunique()

In [ ]:
X = df["text"]
y = df["isalive"]

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform([i for i in df["isalive"]])

In [ ]:
X = []
y = df["isalive"]

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(df["polarity"]).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [ ]:
classifier = RandomForestClassifier(n_estimators=500, random_state=0)  
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)  

In [ ]:
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

In [ ]:
plt.plot(df1["date"], df1["polarity"]

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define the 'classDB' database in Mongo
db = client.projDB
char_collection = db.char_img.find()

In [ ]:
char_name_img_list = []
for char in char_collection:
    char_dict = {}
    char_dict["name"] = char["name"]
    char_dict["img_url"] = char["img_url"]
    char_name_img_list.append(char_dict)

In [ ]:
file1 = "data/khaleesi.csv"
df1 = pd.read_csv(file1)
df = df1[["char", "date", "polarity"]]
df["char"] = "Daenerys Targaryen"    
df2 = df.groupby(["char", "date"])["polarity"].mean()
file_df = df2.reset_index()

In [ ]:
file_df.loc[file_df["char"] == "Daenerys Targaryen"]

In [ ]:
for character in char_name_img_list:
    print (character["name"])
    char_data_df = file_df.loc[file_df["char"] == character["name"]]
    print(char_data_df.size)
    if char_data_df.size > 0:
        character["date"] = [date for date in char_data_df["date"]]
        character["polarity"] = [polarity for polarity in char_data_df["polarity"]]


In [ ]:
char_name_img_list